In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from utils import *

/home/I321267/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
m = 100000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m, locale='zh')

In [3]:
dataset[:10]

[('0点43分53秒', 'ABS>00:43:53'),
 ('52秒后', '+52S        '),
 ('下午11:58:08', 'ABS>23:58:08'),
 ('上午12点50分53秒', 'ABS>00:50:53'),
 ('再过2分钟', '+2M         '),
 ('周六7点55分57秒', 'TW6>07:55:57'),
 ('下午5点2分', 'ABS>17:02:09'),
 ('33小时以后', '+33H        '),
 ('下午8点25分16秒', 'ABS>20:25:16'),
 ('5秒以后', '+5S         ')]

In [4]:
Tx = 30
Ty = 12
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print('X.shape', X.shape)
print('Y.shape', Y.shape)
print('Xoh.shape', Xoh.shape)
print('Yoh.shape', Yoh.shape)

X.shape (100000, 30)
Y.shape (100000, 12)
Xoh.shape (100000, 30, 41)
Yoh.shape (100000, 12, 23)


In [5]:
index = 0
print('Source time:', dataset[index][0])
print('Target time:', dataset[index][1])
print()
print('Source after preprocessing (indices):', X[index])
print('Target after preprocessing (indices):', Y[index])
print()
print('Source after preprocessing (one-hot):', Xoh[index])
print('Target after preprocessing (one-hot):', Yoh[index])

Source time: 0点43分53秒
Target time: ABS>00:43:53

Source after preprocessing (indices): [ 0 34  4  3 22  5  3 35 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40 40
 40 40 40 40 40 40]
Target after preprocessing (indices): [14 15 20 13  2  2 12  6  5 12  7  5]

Source after preprocessing (one-hot): [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0.

In [6]:
repeator = RepeatVector(Tx, name='rep')
concatenator = Concatenate(axis=-1, name='conc')
densor = Dense(1, activation='relu', name='densor')
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes=1, name='doter')

In [7]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor(concat)
    alphas = activator(e)
    context = dotor([alphas, a])
    
    return context

In [8]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True, name='post_activation')
output_layer = Dense(len(machine_vocab), activation=softmax, name='output')

In [9]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model(inputs = [X, s0, c0], outputs = outputs, name='TranslationModel')
    
    return model

In [10]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 41)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       18944       input_1[0][0]                    
__________________________________________________________________________________________________
rep (RepeatVector)              (None, 30, 64)       0           s0[0][0]                         
                                                                 post_activation[0][0]            
          

In [12]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [13]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [14]:
model.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10
100000/100000 [==============================] - 222s 2ms/step - loss: 11.6470 - output_loss: 1.6362 - output_acc: 0.8500 - output_acc_1: 0.7064 - output_acc_2: 0.5384 - output_acc_3: 0.9268 - output_acc_4: 0.7006 - output_acc_5: 0.4375 - output_acc_6: 0.9841 - output_acc_7: 0.4954 - output_acc_8: 0.4333 - output_acc_9: 0.9850 - output_acc_10: 0.4384 - output_acc_11: 0.4001
Epoch 2/10
100000/100000 [==============================] - 209s 2ms/step - loss: 7.7325 - output_loss: 1.4989 - output_acc: 0.9310 - output_acc_1: 0.8446 - output_acc_2: 0.8452 - output_acc_3: 0.9993 - output_acc_4: 0.8228 - output_acc_5: 0.5498 - output_acc_6: 1.0000 - output_acc_7: 0.6522 - output_acc_8: 0.6093 - output_acc_9: 0.9999 - output_acc_10: 0.5109 - output_acc_11: 0.4673
Epoch 3/10
100000/100000 [==============================] - 213s 2ms/step - loss: 6.7465 - output_loss: 1.4632 - output_acc: 0.9434 - output_acc_1: 0.8934 - output_acc_2: 0.9134 - output_acc_3: 0.9999 - output_acc_4: 0.8553 -

In [42]:
examples = ['明天下午3:04', '这周六10:20', '上午9点10分', '10分钟以后', 
            '下周日下午3点45分', '再过3小时', '今天22点08分30秒']    
source = np.array([string2int(example, Tx, human_vocab) for example in examples])
source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
prediction = model.predict([source, s0, c0])
prediction = np.argmax(prediction, axis = -1).transpose()

for i in range(len(examples)):
    print("human:   ", examples[i])
    output = [inv_machine_vocab[int(j)] for j in prediction[i]]
    print("machine: ", ''.join(output))

human:    明天下午3:04
machine:  +3D>14:30:49
human:    这周六10:20
machine:  TW6>10:00:00
human:    上午9点10分
machine:  ABS>09:01:11
human:    10分钟以后
machine:  +10M        
human:    下周日下午3点45分
machine:  NW7>14:45:04
human:    再过3小时
machine:  +3H         
human:    今天22点08分30秒
machine:  +0D>22:08:08
